In [2]:
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules


In [4]:
df = pd.read_excel("Online Retail.xlsx")
df.head()

,InvoiceNo,StockCode,lower,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,white hanging heart t-light holder,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,white metal lantern,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,cream cupid hearts coat hanger,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,knitted union flag hot water bottle,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,red woolly hottie white heart.,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [5]:
# df.shape
df['InvoiceNo'].str.contains('C').sum()

9288

In [6]:
df['Description'] = df['Description'].str.strip()

df.dropna(subset=['InvoiceNo'], inplace=True)

df['InvoiceNo'] = df['InvoiceNo'].astype(str)

df = df[~df['InvoiceNo'].str.contains('C')]

In [7]:
# df['Description'].str.contains("CUTLERY SET")==True
df[df['Description'].str.contains("CUTLERY SET")==True]

,InvoiceNo,StockCode,lower,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
90,536378,84997B,red 3 piece retrospot cutlery set,RED 3 PIECE RETROSPOT CUTLERY SET,12,2010-12-01 09:37:00,3.75,14688.0,United Kingdom
91,536378,84997C,blue 3 piece polkadot cutlery set,BLUE 3 PIECE POLKADOT CUTLERY SET,6,2010-12-01 09:37:00,3.75,14688.0,United Kingdom
468,536408,84997B,red 3 piece retrospot cutlery set,RED 3 PIECE RETROSPOT CUTLERY SET,6,2010-12-01 11:41:00,3.75,14307.0,United Kingdom
1017,536522,84997D,pink 3 piece polkadot cutlery set,PINK 3 PIECE POLKADOT CUTLERY SET,1,2010-12-01 12:49:00,3.75,15012.0,United Kingdom
1052,536522,84997C,blue 3 piece polkadot cutlery set,BLUE 3 PIECE POLKADOT CUTLERY SET,1,2010-12-01 12:49:00,3.75,15012.0,United Kingdom
...,...,...,...,...,...,...,...,...,...
485520,577609,23253,NaN,16 PIECE CUTLERY SET PANTRY DESIGN,4,2011-11-21 09:51:00,15.95,12349.0,Italy
492942,578149,23253,NaN,16 PIECE CUTLERY SET PANTRY DESIGN,1,2011-11-23 11:11:00,33.29,NaN,United Kingdom
497761,578472,23253,NaN,16 PIECE CUTLERY SET PANTRY DESIGN,4,2011-11-24 12:40:00,15.95,12476.0,Germany
532520,580983,23253,NaN,16 PIECE CUTLERY SET PANTRY DESIGN,1,2011-12-06 16:26:00,33.29,NaN,United Kingdom


In [8]:
df.dropna(subset=['Description'], inplace=True)

In [9]:
# df.isnull().sum()
cutlery_items = df[df["Description"].str.contains("CUTLERY SET", case=False)]["Description"].unique()
cutlery_items

array(['RED 3 PIECE RETROSPOT CUTLERY SET',
       'BLUE 3 PIECE POLKADOT CUTLERY SET',
       'PINK 3 PIECE POLKADOT CUTLERY SET',
       'GREEN 3 PIECE POLKADOT CUTLERY SET',
       '16 PIECE CUTLERY SET PANTRY DESIGN',
       '16 PC CUTLERY SET PANTRY DESIGN',
       '18PC WOODEN CUTLERY SET DISPOSABLE'], dtype=object)

In [10]:
df["Description"] = df["Description"].replace(
    {
        'RED 3 PIECE RETROSPOT CUTLERY SET':'CUTLERY SET',
       'BLUE 3 PIECE POLKADOT CUTLERY SET':'CUTLERY SET',
       'PINK 3 PIECE POLKADOT CUTLERY SET':'CUTLERY SET',
       'GREEN 3 PIECE POLKADOT CUTLERY SET':'CUTLERY SET',
       '16 PIECE CUTLERY SET PANTRY DESIGN':'CUTLERY SET',
       '16 PC CUTLERY SET PANTRY DESIGN':'CUTLERY SET',
       '18PC WOODEN CUTLERY SET DISPOSABLE':'CUTLERY SET'
    }
)

In [11]:
df[df["Description"] == "CUTLERY SET"].count()

InvoiceNo      544
StockCode      544
lower            5
Description    544
Quantity       544
InvoiceDate    544
UnitPrice      544
CustomerID     435
Country        544
dtype: int64

In [12]:
def create_basket(country):
    basket = (df[df['Country'] == country]
              .groupby(['InvoiceNo', 'Description'])['Quantity']
              .sum().unstack().reset_index().fillna(0)
              .set_index('InvoiceNo'))

    # Convert to boolean (True for presence, False for absence)
    basket = basket > 0  # This automatically creates a boolean DataFrame
    return basket


In [ ]:
for country in countries:
    print(f"\nAnalysis for {country}:")
    basket = create_basket(country)

#     # Apply the Apriori algorithm with a minimum support threshold of 0.05
#     frequent_itemsets = apriori(basket, min_support=0.02, use_colnames=True)

#     # Generate association rules with a minimum lift threshold of 1
#     rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)

#     # Filter rules that include 'Cutlery Set' in the antecedents or consequents
#     cutlery_rules = rules[
#     rules['antecedents'].apply(lambda x: any('CUTLERY SET' in item.lower() for item in x)) |
#     rules['consequents'].apply(lambda x: any('CUTLERY SET' in item.lower() for item in x))
#     ]


#     # Sort the association rules based on confidence and lift values in descending order
#     cutlery_rules = cutlery_rules.sort_values(by=['confidence', 'lift'], ascending=[False, False])

#     # Print the top association rules involving 'Cutlery Set'
#     for _, rule in cutlery_rules.iterrows():
#         antecedents = ', '.join(rule['antecedents'])
#         consequents = ', '.join(rule['consequents'])
#         print(f"Rule: {antecedents} -> {consequents}")
#         print(f"Support: {rule['support']:.4f}")
#         print(f"Confidence: {rule['confidence']:.4f}")
#         print(f"Lift: {rule['lift']:.4f}\n")

# countries = ["France", "United Kingdom", "Portugal", "Sweden"]    break


Analysis for France:


In [ ]:
countries = ["France", "United Kingdom", "Portugal", "Sweden"]

for country in countries:
    print(f"\nAnalysis for {country}:")
    basket = create_basket(country)

    frequent_itemsets = apriori(basket, min_support=0.01, use_colnames=True)
    # print(frequent_itemsets.head())

    rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
    # print(rules.head())

    cutlery_rules = rules[
    rules['antecedents'].apply(lambda x: any('CUTLERY SET' in item for item in x)) |
    rules['consequents'].apply(lambda x: any('CUTLERY SET' in item for item in x))
    ]
    cutlery_rules = cutlery_rules.sort_values(by=['confidence', 'lift'], ascending=[False, False])

    # print(cutlery_rules.head())
    # Print the top association rules involving 'Cutlery Set'
    for _, rule in cutlery_rules.iterrows():
        antecedents = ', '.join(rule['antecedents'])
        consequents = ', '.join(rule['consequents'])
        print(f"Rule: {antecedents} -> {consequents}")
        print(f"Support: {rule['support']:.4f}")
        print(f"Confidence: {rule['confidence']:.4f}")
        print(f"Lift: {rule['lift']:.4f}\n")

